## Импорт библиотек

In [1]:
from lxml import etree # библиотека для работы с XML-структурами

## Парсинг словаря OpenCorpora

In [2]:
def parse_opencorpora_dict(file_path):
    tree = etree.parse(file_path)
    root = tree.getroot() # получаем корневой элемент XML-дерева

    lemmata = {} # инициализация словаря для хранения лемм
    for lemma in root.findall(".//lemma"): # проходим по всем леммам в словаре
        lemma_element = lemma.find(".//l") # извлекаем следующий элемент леммы

        if lemma_element is not None:
            lemma_text = lemma_element.get('t') # извлекаем словарную форму слова
            pos_elements = lemma.findall(".//g") # извлекаем атрибуты, содержащие грамматическую информацию о слове (часть речи и др.)
            pos = {g.get('v') for g in pos_elements if g is not None}  # сохраняем часть речи (и другие возможные атрибуты) как множество
            forms = [f.get('t') for f in lemma.findall(".//f")] # извлекаем все возможные формы слова (например, склонения, спряжения)

            # связываем каждую форму слова с леммой и частью речи
            for form in forms:
                lemmata[form] = (lemma_text, pos)

    return lemmata # функция возвращает словарь, где ключ — форма слова, а значение — кортеж с леммой и частью речи

## Лемматизация текста

Функция для извлечения части речи из атрибутов слова (тегов).

In [ ]:
def extract_part_of_speech(tags):
    # множество всех возможных тегов частей речи
    pos_map = {
        'NOUN': 'NOUN',  # Существительное
        'ADJF': 'ADJ',   # Прилагательное
        'VERB': 'VERB',  # Глагол
        'ADV': 'ADV',    # Наречие
        'PRCL': 'PRCL',  # Частица
        'PR': 'PR',      # Предлог
        'NUMR': 'NUM',   # Числительное
        'NPRO': 'NPRO',  # Местоимение
        'ADVB': 'ADVB',  # Наречие
        'PREP': 'PREP'   # Предлог
    }

    # проходим по тегам и ищем совпадение среди тегов из множества выше
    for tag in tags:
        if tag in pos_map:
            return pos_map[tag] # функция возвращает часть речи в случае, если тег нашел совпадение в словаре тегов частей речи

    return 'UNKNOWN' # функция возвращает строку 'неизвестно', если части речи не была обнаружена среди тегов

In [3]:
def extract_part_of_speech(tags):
    # множество всех возможных тегов частей речи
    pos_tags = {
        'NOUN',  # Существительное
        'ADJF',  # Прилагательное
        'VERB',  # Глагол
        'ADV',   # Наречие
        'PRCL',  # Частица
        'PR',    # Предлог
        'NUMR',  # Числительное
        'NPRO',  # Местоимение
        'ADVB',  # Наречие
        'PREP'   # Предлог
    }

    # Проходим по тегам и ищем совпадение среди тегов из множества
    for tag in tags:
        if tag in pos_tags:
            return tag  # Возвращаем тег, если нашли совпадение

    return 'UNKNOWN'  # Возвращаем 'UNKNOWN', если часть речи не была обнаружена среди тегов

In [4]:
def lemmatize_text(text, lemmata):
    tokens = text.split() # токенизация текста (разбивка на отдельные слова)
    lemmatized_words = [] # инициализация множества для хранения лемматизированных слов

    for token in tokens: # проходим по каждому токену
        token = token.lower().replace('ё', 'е').strip('.,!?')  # нормализация
        if token in lemmata: # проверяем, есть ли слово в словаре лемм
            lemma, pos = lemmata[token] # получаем лемму и множество тегов
            part_of_speech = extract_part_of_speech(pos) # извлекаем часть речи из тегов
            lemmatized_words.append(f"{token}{{{lemma}={part_of_speech}}}") # формируем результат для каждого токена
        else:
            lemmatized_words.append(f"{token}{{UNKNOWN=UNKNOWN}}")

    return ' '.join(lemmatized_words) # функция возвращает текст с лемматизированными словами

## main

In [5]:
lemmata_dict = parse_opencorpora_dict('dict.opcorpora.xml')

In [6]:
text = "Стала стабильнее экономическая и политическая обстановка, предприятия вывели из тени зарплаты сотрудников."
print(lemmatize_text(text, lemmata_dict))

стала{стал=VERB} стабильнее{стабильнее=UNKNOWN} экономическая{экономический=ADJF} и{и=NOUN} политическая{политический=ADJF} обстановка{обстановка=NOUN} предприятия{предприятие=NOUN} вывели{вывел=VERB} из{иза=NOUN} тени{тень=NOUN} зарплаты{зарплата=NOUN} сотрудников{сотрудник=NOUN}


In [7]:
text = "Все Гришины одноклассники уже побывали за границей, он был чуть ли не единственным, кого не вывозили никуда дальше Красной Пахры."
print(lemmatize_text(text, lemmata_dict))

все{весь=ADJF} гришины{гришин=ADJF} одноклассники{одноклассник=NOUN} уже{уже=PRCL} побывали{побывал=VERB} за{за=PREP} границей{граница=NOUN} он{он=NPRO} был{есть=VERB} чуть{чуть=UNKNOWN} ли{ли=NOUN} не{не=PRCL} единственным{единственный=ADJF} кого{кто=NPRO} не{не=PRCL} вывозили{вывозил=VERB} никуда{никуда=ADVB} дальше{дальше=UNKNOWN} красной{красный=ADJF} пахры{пахра=NOUN}
